In [1]:
import pickle
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv("task2_allData.csv")
test_public = pd.read_csv("task2_public_test_data.csv")
test = pd.read_csv("task2_private_test_data.csv")

In [3]:
train = pd.merge(train, test_public, how = "outer")

In [4]:
x_train = train.drop(["AnswerValue", "AnswerId"], axis=1)
y_train = train.AnswerValue
x_test = test.drop(["AnswerValue", "AnswerId"], axis=1)
y_test = test.AnswerValue

In [5]:
y_train = y_train.apply(lambda x: x - 1)
y_test = y_test.apply(lambda x: x - 1)

In [6]:
xgb_model = xgb.XGBRegressor(
    objective="multi:softmax",
    random_state=0,
    num_class=4,
    n_estimators=50,
    max_depth=12,
    )

In [7]:
xgb_model = xgb_model.fit(x_train, y_train)

[12:54:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [8]:
y_predict = xgb_model.predict(x_test)

In [9]:
accuracy_score(y_test, y_predict)

0.653280947344115

In [10]:
print(classification_report(y_test, y_predict, target_names=["A", "B", "C", "D"]))

              precision    recall  f1-score   support

           A       0.68      0.64      0.66    478895
           B       0.64      0.66      0.65    517881
           C       0.64      0.67      0.66    524238
           D       0.66      0.64      0.65    462468

    accuracy                           0.65   1983482
   macro avg       0.65      0.65      0.65   1983482
weighted avg       0.65      0.65      0.65   1983482



In [12]:
df = pd.DataFrame(y_predict, columns =['XGBoost'])
df.to_csv("XGBoost.csv", header=True, index=False)

In [16]:
pickle.dump(xgb_model, open("./models/XGB.sav", 'wb'))